# Importando bibliotecas

In [5]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from arch.unitroot import PhillipsPerron as pp
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.simplefilter('ignore')
%matplotlib inline
sns.set_style("whitegrid")
plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)

# 1. Importando base de dados formatada

## 1.1 Tratando os dados internos

In [6]:
dt_parse = lambda dates: dt.strptime(dates, '%Y-%m-%d')

data = pd.read_csv('data/transformed/2000_2021_Brazilian_Northeast_Region_daily_MW_instantaneous_maximum_demand_series.csv',
                   header=0, names=['index','value', 'date'], parse_dates=True, index_col='index', date_parser=dt_parse)
data['value'] = data['value'].astype(float)
data['date'] = data['date'].astype('datetime64')
data = data[data['value'] > 0.0]
data.head()

,value,date
index,,
2000-01-01,6205.0,2000-01-01
2000-01-02,6797.0,2000-01-02
2000-01-03,7024.0,2000-01-03
2000-01-04,6921.0,2000-01-04
2000-01-05,7025.0,2000-01-05


# 2. SARIMA

## 2.1 Tratando os dados internos

In [7]:
split_date = '1/1/2018 12:00:00 AM'
test  = data[split_date : ]
train = data[ : split_date]

train = train['value']
test = test['value']
series = pd.concat([train, test], axis=0)

series

index
2000-01-01     6205.000
2000-01-02     6797.000
2000-01-03     7024.000
2000-01-04     6921.000
2000-01-05     7025.000
                ...    
2021-01-27    12308.892
2021-01-28    12550.095
2021-01-29    12413.574
2021-01-30    11988.032
2021-01-31    11540.266
Name: value, Length: 7702, dtype: float64

## 2.2 Métodos para treinamento

In [8]:
# Mostrar previsão de acordo com os dados originais
def plot_forecasts(series, forecasts, test):
	plt.plot(series) # blue

	# Colocando no gráfico as previsões com a cor vermelha
	for i in range(len(forecasts)):
        # Inicio e fim dos dos deslocamentos por x
		off_s = len(series) - len(test) + i - 1
		off_e = off_s + len(forecasts[i]) + 1
		xaxis = [x for x in range(off_s, off_e)]

        # Valores dos multiplos passos de previsão
		yaxis = [series[off_s]] + forecasts[i]
		plt.plot(xaxis, yaxis, color='red')

	# Gráfico
	plt.show()

# Avaliação de erro do SARIMA
def evaluate_forecasts(test, forecasts, n_steps):
    sum_rmse = 0
    sum_mape = 0
    for i in range(n_steps):
        ypred_ts = [forecast[i] for forecast in forecasts]
        ytrue_ts = test[i:len(ypred_ts)+i]
        # Calculando os erros
        mape = mean_absolute_percentage_error(ytrue_ts, ypred_ts)*100
        rmse = sqrt(mean_squared_error(ytrue_ts, ypred_ts))
        sum_rmse += rmse
        sum_mape += mape
        print(f"t+{(i+1)} RMSE: {rmse} | MAPE: {mape}")

    # média dos erros
    print(f"Média do RMSE: {sum_rmse/n_steps}")
    print(f"Média do MAPE: {sum_mape/n_steps}")

# Treinamento de previsão por ARIMA por multiplos passos
def sarima_forecast(train, config, n_steps):
	order, s_order, trend = config
	# Definindo e moldando o modelo
	model = model = SARIMAX(train, order=order, seasonal_order=s_order, trend=trend, enforce_stationarity=False,
                            enforce_invertibility=False)
	model_fit = model.fit()

	# Fazendo previsão com multiplos passos
	yhat = model_fit.predict(len(train), len(train)+n_steps-1)
	yhat_list = [value for value in yhat]

	return yhat_list

## 2.3 Treinamento e resultado

In [ ]:
test = test.values
series = series.values
train = [x for x in train]

n_steps = 7

# Ordem de tendencial
p_param = 4
d_param = 1
q_param = 13
# Periodo e ordem sazonal
P_param = 4
D_param = 1
Q_param = 2
m_param = 12
# Tipo de tendencia
t_params = 'ct'  # outros tipos: 'n','c','t','ct'
# configuração
config = [(p_param,d_param,q_param), (P_param,D_param,Q_param,m_param), t_params]

# lista para armazenar os paços de predição e a quantidade de predições a frente
forecasts = list()
n_forecasts = len(test) - n_steps + 1

for i in range(n_forecasts):
    # prevendo valor por sarima
	yhat = sarima_forecast(train, config, n_steps)
    # adicionando valor de predição a lista
	forecasts.append(yhat)
	# Adicionando valor de previsão pelo treino para o proximo loop
	obs = test[i]
	train.append(obs)

# Avaliação da previsão
evaluate_forecasts(test, forecasts, n_steps)
# Mostrar previsão
plot_forecasts(series, forecasts, test)